### Topic Modelling NMF:

In [1]:
import twint
import pandas as pd
pd.options.display.max_columns = 100
from glob import glob
import os

import matplotlib.pyplot as plt
import seaborn as sns

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re

import string
re.compile('<title>(.*)</title>')
import nest_asyncio
nest_asyncio.apply()


from itertools import chain
import collections
from collections import Counter
import gensim
from gensim import corpora, models, similarities
import pyLDAvis.gensim_models
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer  

from gensim.corpora import Dictionary
from gensim.models import TfidfModel
from gensim.models.nmf import Nmf
from gensim.models import LdaModel
from gensim.models.coherencemodel import CoherenceModel
from operator import itemgetter # Already part of base Python
from sentence_transformers import SentenceTransformer

C:\Users\Sai Thejaswin\anaconda3\lib\site-packages\sklearn\decomposition\_lda.py:28: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  EPS = np.finfo(np.float).eps


In [71]:
nouns_1 = pd.read_csv(r'C:\Users\Sai Thejaswin\OneDrive\Documents\MS\GSU\Sprint_Project\Data\nouns_phrases_1.csv')
nouns_1['nouns']=nouns_1['nouns'].astype('str')
nouns_1['nouns_embed'] = nouns_1['nouns'].apply(lambda x : x.replace(' ', '_'))
nouns_1['tokenized'] = nouns_1.apply(lambda row: nltk.word_tokenize(row['nouns_embed']), axis=1)
nouns_1_list = nouns_1['tokenized'].tolist()
dictionary_1 = Dictionary(nouns_1_list)
dictionary_1.filter_extremes(no_below=5, no_above=0.7)
corpus_1 = [dictionary_1.doc2bow(document) for document in nouns_1_list]
tfidf_gensim_1 = TfidfModel(corpus_1)
vectors_gensim_1 = tfidf_gensim_1[corpus_1]

In [107]:
# Run NMF model; try 10 topics for now
model_nmf = Nmf(vectors_gensim_1, num_topics = 5, id2word = dictionary_1, kappa = 0.1,random_state = 42)

In [74]:
# Word clouds per topic
for t in range(model_nmf.num_topics):
    wordcloud = WordCloud(
        width = 750, 
        height = 500,
        max_words = 200, 
        background_color = 'white').fit_words(dict(model_nmf.show_topic(t, 200)))
    wordcloud.recolor(color_func = lambda *args, **kwargs: 'black')
    plt.figure(figsize=[10,10])
    plt.imshow(wordcloud)
    plt.axis("off")
    plt.title("Topic #" + str(t))
    plt.show()

In [108]:
# Reorganize as dataframe for easier viewing
topics_nmf = pd.DataFrame(model_nmf.show_topics())
topics_nmf = topics_nmf[1].str.split('+', n = 10, expand = True)
topics_nmf = topics_nmf.replace(r'\d', '', regex=True)
topics_nmf = topics_nmf.replace(r'\*', '', regex=True)
topics_nmf = topics_nmf.replace(r'\"', '', regex=True)
topics_nmf = topics_nmf.replace(r'\.', '', regex=True)
topics_nmf.columns = ['word_1', 'word_2', 'word_3', 'word_4', 'word_5'
                      ,'word_6', 'word_7', 'word_8', 'word_9', 'word_10']
topics_nmf = topics_nmf.transpose()
topics_nmf.columns = ['topic_1', 'topic_2', 'topic_3', 'topic_4', 'topic_5']
                      #'topic_6', 'topic_7', 'topic_8', 'topic_9', 'topic_10']

topics_nmf

,topic_1,topic_2,topic_3,topic_4,topic_5
word_1,information,covid-,learn,covid,ai
word_2,mental_health,covid,nhs,boozallen,icu
word_3,,icu,va,mental_health,digitalhealth
word_4,please,covid,electronic,download,hie
word_5,uk,unprecedented_disruptions,data,cybersecurity,covid
word_6,digital_health,apple,digital,uk,app_workout
word_7,public_health,find,youtube,learn_boozallen_epimaps,please
word_8,coronavirus,solving,electronic_case,talent_health_analytics_cyber,telehealth_remain
word_9,'ll,join,digitalhealth,april,china
word_10,covid,health_app,may,new_cases,toughest_problems


# Phase-2

In [109]:
nouns_2 = pd.read_csv(r'C:\Users\Sai Thejaswin\OneDrive\Documents\MS\GSU\Sprint_Project\Data\nouns_phrases_2.csv')
nouns_2['nouns']=nouns_2['nouns'].astype('str')
nouns_2['nouns_embed'] = nouns_2['nouns'].apply(lambda x : x.replace(' ', '_'))
nouns_2['tokenized'] = nouns_2.apply(lambda row: nltk.word_tokenize(row['nouns_embed']), axis=1)
nouns_2_list = nouns_2['tokenized'].tolist()
dictionary_2 = Dictionary(nouns_2_list)
dictionary_2.filter_extremes(no_below=5, no_above=0.7)
corpus_2 = [dictionary_2.doc2bow(document) for document in nouns_2_list]
tfidf_gensim_2 = TfidfModel(corpus_2)
vectors_gensim_2 = tfidf_gensim_2[corpus_2]
# Run NMF model; try 10 topics for now
model_nmf_2 = Nmf(vectors_gensim_2, num_topics = 5, id2word = dictionary_2, kappa = 0.1, eval_every = 10, random_state = 42)


# Reorganize as dataframe for easier viewing
topics_nmf_2 = pd.DataFrame(model_nmf_2.show_topics())
topics_nmf_2 = topics_nmf_2[1].str.split('+', n = 10, expand = True)
topics_nmf_2 = topics_nmf_2.replace(r'\d', '', regex=True)
topics_nmf_2 = topics_nmf_2.replace(r'\*', '', regex=True)
topics_nmf_2 = topics_nmf_2.replace(r'\"', '', regex=True)
topics_nmf_2 = topics_nmf_2.replace(r'\.', '', regex=True)
topics_nmf_2.columns = ['word_1', 'word_2', 'word_3', 'word_4', 'word_5',
                      'word_6', 'word_7', 'word_8', 'word_9', 'word_10']
topics_nmf_2 = topics_nmf_2.transpose()
topics_nmf_2.columns = ['topic_1', 'topic_2', 'topic_3', 'topic_4', 'topic_5']
                      #'topic_6', 'topic_7', 'topic_8', 'topic_9', 'topic_10']

topics_nmf_2

,topic_1,topic_2,topic_3,topic_4,topic_5
word_1,covid,information,public_health,covid-,covid
word_2,,ai,learn,icu,icu
word_3,information,learn,medical_things,neuralink,remote_patient
word_4,rtされた数:,r_eicu_ツイート数:,surveillance_covid-_data_analytics_ai,august,covid-_electronic_case_reporting
word_5,internet,covid,join,mental,healthtech
word_6,read,trump,mental_health,september,onc
word_7,iomt,singapore,health_technology,digitalhealth,internet
word_8,us,belgium_icu,providertech_healthtech_healthcare_pophealth_...,r_eicu_ツイート数:,health_system
word_9,electronic,electronic_tags_monitor_travellers,improving,health_information_exchange,please
word_10,remote_patient,canada,mh,femtech,coronavirus


# Phase-3

In [110]:
nouns_3 = pd.read_csv(r'C:\Users\Sai Thejaswin\OneDrive\Documents\MS\GSU\Sprint_Project\Data\nouns_phrases_3.csv')
nouns_3['nouns']=nouns_3['nouns'].astype('str')
nouns_3['nouns_embed'] = nouns_3['nouns'].apply(lambda x : x.replace(' ', '_'))
nouns_3['tokenized'] = nouns_3.apply(lambda row: nltk.word_tokenize(row['nouns_embed']), axis=1)
nouns_3_list = nouns_3['tokenized'].tolist()
dictionary_3 = Dictionary(nouns_3_list)
dictionary_3.filter_extremes(no_below=5, no_above=0.7)
corpus_3 = [dictionary_3.doc2bow(document) for document in nouns_3_list]
tfidf_gensim_3 = TfidfModel(corpus_3)
vectors_gensim_3 = tfidf_gensim_3[corpus_3]
# Run NMF model; try 10 topics for now
model_nmf_3 = Nmf(vectors_gensim_3, num_topics = 5, id2word = dictionary_3, kappa = 0.1, eval_every = 10, random_state = 42)


# Reorganize as dataframe for easier viewing
topics_nmf_3 = pd.DataFrame(model_nmf_3.show_topics())
topics_nmf_3 = topics_nmf_3[1].str.split('+', n = 10, expand = True)
topics_nmf_3 = topics_nmf_3.replace(r'\d', '', regex=True)
topics_nmf_3 = topics_nmf_3.replace(r'\*', '', regex=True)
topics_nmf_3 = topics_nmf_3.replace(r'\"', '', regex=True)
topics_nmf_3 = topics_nmf_3.replace(r'\.', '', regex=True)
topics_nmf_3.columns = ['word_1', 'word_2', 'word_3', 'word_4', 'word_5',
                      'word_6', 'word_7', 'word_8', 'word_9', 'word_10']
topics_nmf_3 = topics_nmf_3.transpose()
topics_nmf_3.columns = ['topic_1', 'topic_2', 'topic_3', 'topic_4', 'topic_5']
                     # 'topic_6', 'topic_7', 'topic_8', 'topic_9', 'topic_10']

topics_nmf_3

,topic_1,topic_2,topic_3,topic_4,topic_5
word_1,learn,ai,information,april,covid-
word_2,covid,hie,may,ai,icu
word_3,read,covid-,feasibility_analysis_syndromic_surveillance,please,
word_4,covid,artificial_intelligence,health_information_exchange,algorithm,uk
word_5,join,find,public_health,visit,digitaldivide
word_6,digital,algorithms,algorithm,electronic_health_records,
word_7,please,biden,role,twitch,safety
word_8,twitter,role,dna,healthcare,drvanessawalker
word_9,india,epidemiological_surveillance,covid,twitchsupport,teamsutter
word_10,icu,health_care,youtube,california,treat


# Phase-4

In [111]:
nouns_4 = pd.read_csv(r'C:\Users\Sai Thejaswin\OneDrive\Documents\MS\GSU\Sprint_Project\Data\nouns_phrases_4.csv')
nouns_4['nouns']=nouns_4['nouns'].astype('str')
nouns_4['nouns_embed'] = nouns_4['nouns'].apply(lambda x : x.replace(' ', '_'))
nouns_4['tokenized'] = nouns_4.apply(lambda row: nltk.word_tokenize(row['nouns_embed']), axis=1)
nouns_4_list = nouns_4['tokenized'].tolist()
dictionary_4 = Dictionary(nouns_4_list)
dictionary_4.filter_extremes(no_below=5, no_above=0.7)
corpus_4 = [dictionary_4.doc2bow(document) for document in nouns_4_list]
tfidf_gensim_4 = TfidfModel(corpus_4)
vectors_gensim_4 = tfidf_gensim_4[corpus_4]
# Run NMF model; try 10 topics for now
model_nmf_4 = Nmf(vectors_gensim_4, num_topics = 5, id2word = dictionary_4, kappa = 0.1, eval_every = 10, random_state = 42)


# Reorganize as dataframe for easier viewing
topics_nmf_4 = pd.DataFrame(model_nmf_4.show_topics())
topics_nmf_4 = topics_nmf_4[1].str.split('+', n = 10, expand = True)
topics_nmf_4 = topics_nmf_4.replace(r'\d', '', regex=True)
topics_nmf_4 = topics_nmf_4.replace(r'\*', '', regex=True)
topics_nmf_4 = topics_nmf_4.replace(r'\"', '', regex=True)
topics_nmf_4 = topics_nmf_4.replace(r'\.', '', regex=True)
topics_nmf_4.columns = ['word_1', 'word_2', 'word_3', 'word_4', 'word_5',
                      'word_6', 'word_7', 'word_8', 'word_9', 'word_10']
topics_nmf_4 = topics_nmf_4.transpose()
topics_nmf_4.columns = ['topic_1', 'topic_2', 'topic_3', 'topic_4', 'topic_5']
                      #'topic_6', 'topic_7', 'topic_8', 'topic_9', 'topic_10']

topics_nmf_4

,topic_1,topic_2,topic_3,topic_4,topic_5
word_1,covid-,ai,healthcare,information,ai
word_2,read,date,epic_anthem_strike_deal_support,covid,may
word_3,usd,read,electronic,,ontario
word_4,victoria,digital,health_technology,unknown_wallet,l_biozhena_files_downloads
word_5,btc,june,vc,vic,obgyn
word_6,nsw,ehr,eu,ps,learn
word_7,digitalhealth,unknown_wallet,international_health_week_review,digital_digitalhealth_health,precisionmedicine_womenshealth_fertilityaware...
word_8,blockchain,data,scripps,please,
word_9,qr,btc,contact,digital_health,nps
word_10,nhs,rt,obgyn,electronic,vicgovdh


# Phase 5

In [112]:
nouns_5 = pd.read_csv(r'C:\Users\Sai Thejaswin\OneDrive\Documents\MS\GSU\Sprint_Project\Data\nouns_phrases_5.csv')
nouns_5['nouns']=nouns_5['nouns'].astype('str')
nouns_5['nouns_embed'] = nouns_5['nouns'].apply(lambda x : x.replace(' ', '_'))
nouns_5['tokenized'] = nouns_5.apply(lambda row: nltk.word_tokenize(row['nouns_embed']), axis=1)
nouns_5_list = nouns_5['tokenized'].tolist()
dictionary_5 = Dictionary(nouns_5_list)
dictionary_5.filter_extremes(no_below=5, no_above=0.7)
corpus_5 = [dictionary_5.doc2bow(document) for document in nouns_5_list]
tfidf_gensim_5 = TfidfModel(corpus_5)
vectors_gensim_5 = tfidf_gensim_5[corpus_5]
# Run NMF model; try 10 topics for now
model_nmf_5 = Nmf(vectors_gensim_5, num_topics = 5, id2word = dictionary_5, kappa = 0.1, eval_every = 10, random_state = 42)


# Reorganize as dataframe for easier viewing
topics_nmf_5 = pd.DataFrame(model_nmf_5.show_topics())
topics_nmf_5 = topics_nmf_5[1].str.split('+', n = 10, expand = True)
topics_nmf_5 = topics_nmf_5.replace(r'\d', '', regex=True)
topics_nmf_5 = topics_nmf_5.replace(r'\*', '', regex=True)
topics_nmf_5 = topics_nmf_5.replace(r'\"', '', regex=True)
topics_nmf_5 = topics_nmf_5.replace(r'\.', '', regex=True)
topics_nmf_5.columns = ['word_1', 'word_2', 'word_3', 'word_4', 'word_5',
                      'word_6', 'word_7', 'word_8', 'word_9', 'word_10']
topics_nmf_5 = topics_nmf_5.transpose()
topics_nmf_5.columns = ['topic_1', 'topic_2', 'topic_3', 'topic_4', 'topic_5']
                      #'topic_6', 'topic_7', 'topic_8', 'topic_9', 'topic_10']

topics_nmf_5

,topic_1,topic_2,topic_3,topic_4,topic_5
word_1,digitalhealth,ai,,mental,internet
word_2,rt,electronic,record,learn,read
word_3,mental_health,rtされた数:,us,iot,cybersecurity
word_4,god,,artificial_intelligence,artificial_intelligence,records_ehrs
word_5,california,immigrants_tuberculosis,前日比:_受け取ったリプライ数:_前日比:_いいねされた数:_前日比:,,medical_technology
word_6,va,digital,read,music_history,rt
word_7,前日比:_受け取ったリプライ数:_前日比:_いいねされた数:_前日比:,source_electronic,records,artificialintelligence,rtされた数:
word_8,barcelona,前日比:_受け取ったリプライ数:_前日比:_いいねされた数:_前日比:,care,vr,learn
word_9,records,cybersecurity,gps,electronic_devices,immigrants_tuberculosis
word_10,ministry,barcelona,seattle,telemedicine,diagnostic
